# Turkish News Category Classification (4. Prediction)
## Import Packages

In [2]:
from scipy.sparse import hstack
import pandas as pd
import pickle
import re
import string
from collections import defaultdict

In [3]:
#  load word and char tfidf vectors
PATH_ROOT = r'../' 
PATH_STOPWORDS_TR = PATH_ROOT + 'data/stopwords_tr_interpress.pkl'
PATH_BLACKLIST_TR = PATH_ROOT + 'data/tr-blacklist.pkl'

#  load dataset
def load_data(path):
    with open(PATH_ROOT + path, 'rb') as handle:
        dataset = pickle.load(handle)
    return dataset

In [4]:
labels = {
    0 : "Kültür-Sanat",
    1 : "Ekonomi",
    2 : "Siyaset",
    3 : "Eğitim",
    4 : "Dünya",
    5 : "Spor",
    6 : "Teknoloji",
    7 : "Magazin",
    8 : "Sağlık",
    9 : "Gündem"
}

## Load TF-IDF Word&Char Vectors

In [5]:
with open(PATH_ROOT + 'vectors/interpress_news_category_tr_lite_train_cleaned_tfidf_word_2000.pkl', 'rb') as handle:
    word_tfidf = pickle.load(handle)

with open(PATH_ROOT + 'vectors/interpress_news_category_tr_lite_train_cleaned_tfidf_char_2000.pkl', 'rb') as handle:
    char_tfidf = pickle.load(handle)

## Load SVM Model

In [6]:
#  load model
with open(PATH_ROOT + 'models/interpress_news_category_tr_lite_classifier_svm_model_4000.sav', 'rb') as handle:
    model = pickle.load(handle)

# PreProcessing

In [7]:
#  load stopwords
def get_stopwords():
    try:
        with open(PATH_STOPWORDS_TR, 'rb') as data_file:
            return pickle.load(data_file)
    except IOError as exc:
        raise IOError("No such stopwords file! Error: " + str(exc))

#  load blacklist
def get_blacklist():
    try:
        with open(PATH_BLACKLIST_TR, 'rb') as data_file:
            return pickle.load(data_file)
    except IOError as exc:
        raise IOError("No such stopwords file! Error: " + str(exc))
        

#  cleaning stopwords
def clean_stopwords(content):
    # content: str
    content = content.split(" ")
    filtered_list = []
    stopwords = get_stopwords()
    for word in content:
        if word not in stopwords:
            filtered_list.append(word)

    text = ' '.join(filtered_list)
    return text

#  cleaning blacklist
def clean_blacklist(content):
    # content: str
    # return: str
    content = content.split(" ")
    filtered_list = []
    blacklist = get_blacklist()
    for word in content:
        if word not in blacklist:
            filtered_list.append(word)

    text = ' '.join(filtered_list)
    return text

#  cleaning URLs
def clean_url(content):
    #  content: str
    #  return: str
    reg_url = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    pattern_url = re.compile(reg_url)
    content = pattern_url.sub('',content)
    return content
    
#  cleaning e-mails
def clean_email(content):
    reg_email='\S*@\S*\s?'
    pattern_email = re.compile(reg_email)
    content = pattern_email.sub('',content)
    return content
    
#  cleaning punctuation
def clean_punctuation(content):
    #regex = re.compile('[%s]' % re.escape(string.punctuation))
    #content = regex.sub(" ", content)
    content = content.translate(content.maketrans("", "", string.punctuation))
    return content
    
#  cleaning digits
def clean_numbers(content):
    remove_digits = str.maketrans('', '', string.digits)
    text = content.translate(remove_digits)
    return text

def listToString(text):
    #  text: string
    #  return: string 
    str1 = " "  
    return (str1.join(text)) 

#  cleaning postfix
#  for example: Venezuela'nın ==> Venezuela
def clean_postfix(content):
    #  content: list
    #  return: str
    spesific_punctation = [39, 8217] # ascii codes of spesific punctations
    all_words = defaultdict(list)
    counter = 0
    for i, word in enumerate(content):
        if len(word) > 2:
            if chr(8217) in word:
                replaced_word = word.replace(chr(8217), " ")  #  clean for ’
                replaced_word = replaced_word.split(" ")[0]
                all_words[i].append(replaced_word)
            elif chr(39) in word:
                replaced_word = word.replace(chr(39), " ")  # clean for '
                replaced_word = replaced_word.split(" ")[0]
                all_words[i].append(replaced_word)
            else:
                all_words[i].append(word)
    all_words = sorted(all_words.items())

    text = ""
    for i in range(len(all_words)):
        text = text + " " + all_words[i][1][0]

    return text

def clean_text(content):
    #  text: string
    #  return: string
    content = content.lower()
    cleaned_email = clean_email(content)
    cleaned_email_url = clean_url(cleaned_email)
    cleaned_email_url = listToString(cleaned_email_url.split("."))
    cleaned_email_url = cleaned_email_url.split(" ")
    cleaned_email_url_postfix = clean_postfix(cleaned_email_url)
    cleaned_email_url_postfix_punctuation = clean_punctuation(cleaned_email_url_postfix)
    cleaned_email_url_postfix_punctuation_numbers = clean_numbers(cleaned_email_url_postfix_punctuation)
    cleaned_email_url_postfix_punctuation_numbers_stopwords = clean_stopwords(cleaned_email_url_postfix_punctuation_numbers)
    cleaned_email_url_postfix_punctuation_numbers_stopwords_blacklist = clean_blacklist(cleaned_email_url_postfix_punctuation_numbers_stopwords)
   
    filtered_sentence = []
    for word in cleaned_email_url_postfix_punctuation_numbers_stopwords_blacklist.split(" "):
        if len(word) > 2:
            filtered_sentence.append(word)
    
    text = ' '.join(filtered_sentence)
    return text


# Custom prediction

### Sample 1

In [28]:
#news = r"Çin biyoteknoloji şirketleri China National Pharmaceutical Group (Sinopharm) ve Chongqing Zhifei Biological Products'ın bir yan kuruluşu tarafından geliştirilen iki corona virüs aşının Güney Afrika mutasyonuna karşı bağışıklığı tetiklediği açıklandı. BioRxiv adlı dergide ön baskısı yayımlanan laboratuvar araştırmasında, aşıyı yaptıran kişilerin kan örnekleri analiz edildi. Araştırmacılar, aşının tetiklediği antikorların Güney Afrika varyantına karşı nötrleştirme aktivitesini koruduğunu söyledi. Makale, Sinopharm'a bağlı Pekin Biyolojik Ürünler Enstitüsü, Çin Bilimler Akademisi Mikrobiyoloji Enstitüsü araştırmacıları tarafından yazıldı. AŞILARIN ETKİNLİĞİ DÜŞTÜ Bununla birlikte,  alınan örneklerdeki antikorların mutant virüse karşı aktivitesinin  orijinal virüse göre daha zayıf olduğu ifade edildi. Bilim insanları, aktivite azalmasının aşıların klinik etkililiğine olan etkisi dikkate alınmalıdır açıklamasını yaptı. Ancak, aşılardaki etkinliğin hangi oranda düştüğü belirtilmedi. Öte yandan, yüzde 79 oranında  etkili olduğu açıklanan Sinopharm aşısı Çin'de genel kullanım için onaylandı  ve Birleşik Arap Emirlikleri (BAE) de  dahil olmak üzere diğer birçok ülkede de kullanıllıyor. "
#news = "Şu sıralar 'Çukur' dizisinde 'Vartolu Sadettin' karakteriyle ekranlarda boy gösteren oyuncu Erkan Kolçak Köstendil ve meslektaşı Cansu Tosun 2018 yılında nikah masasına oturmuştu. Çift şu sıralar bebek heyecanı yaşıyor.Cansu Tosun 4.5 aylık hamile. Cansu Tosun ve Erkan Kolçak Köstendil'in bebeklerinin adına karar verdi."
#news = "Batı dünyasında çok sık rastlanılan, bebek ölümlerine neden olan genetik hastalık SMA'ya, son yıllarda ülkemizde de sık rastlanılmaya başladı. Edinburgh Üniversitesi'nde yapılan deneye göre her 6 bin doğumdan birinde görülen SMA tanısı %95 delesyon tanısı olarak DNA testi sonucuyla konuyor. Geriye kalan %5 oranındaki bozukluk diğer hasar veren mutasyonlar şeklinde gelişebiliyor. 6000 – 10000 doğumda bir görülebilen hastalıkta 40 çocuktan biri taşıyıcı olabiliyor. Her iki ebeveynden bozuk gen çocuğa geçtiğinde ancak çocuk SMA hastası olabiliyor. Sadece anneden veya sadece babadan bozuk gen aktarımı çocukta hastalık oluşturmuyor, ancak taşıyıcılık oluşturabiliyor.."
news = "Faizsiz konut ve taşıt edindiren şirketlere yönelik yasa teklifi Meclis’e sunuldu. 100 milyon lira sermaye şartının etkisi ve BDDK incelemeleri sonrasında firmaların büyük kısmının tasfiye edilmesi, müşterilerin kurumsal firmalara aktarılması bekleniyor."
#news = "PSG'nin dünyaca ünlü yıldız futbolcusu Neymar, sakatlığı sebebiyle 4 hafta sahalardan uzak kalacak. Neymar'ın son yıllarda kız kardeşi Rafaella Santos'un doğum gününe denk gelen tarihlerde sakatlanması dikkat çekiyor..."
#news = "Brezilyalı futbolcu, son yıllarda kardeşinin doğum günü tarihine gelecek şekilde sakatlanıyor. Senaryo, bu yıl da değişmedi. 4 hafta sahalardan uzak kalacak olan Neymar, bir kez daha kız kardeşinin doğum gününde sahalarda olmayacak.2015 ve 2016 yıllarında 11 Mart tarihlerinde kart cezalısı olduğu için Brezilya'da olan Neymar, 2017, 2018, 2019, 2020 ve 2021'de ise bu tarihlerde sakatlık geçirdi. Bu tarihlerde sakat veya cezalı olduğu için Brezilya'ya giden Neymar, bu yıl da Rafaella'nın doğum gününde Brezilya'da olacak." 
cleaned_news = clean_text(news)
cleaned_news

'faizsiz konut taşıt edindiren şirketlere yönelik yasa teklifi meclis sunuldu lira sermaye şartının etkisi bddk incelemeleri firmaların büyük kısmının tasfiye müşterilerin kurumsal firmalara aktarılması bekleniyor'

In [29]:
news

'Faizsiz konut ve taşıt edindiren şirketlere yönelik yasa teklifi Meclis’e sunuldu. 100 milyon lira sermaye şartının etkisi ve BDDK incelemeleri sonrasında firmaların büyük kısmının tasfiye edilmesi, müşterilerin kurumsal firmalara aktarılması bekleniyor.'

## Preprocessing

In [30]:
#  preprocessing
word_transformed_news = word_tfidf.transform([cleaned_news])
char_transformed_news = char_tfidf.transform([cleaned_news])
news_featured = hstack([word_transformed_news, char_transformed_news])

In [31]:
labels

{0: 'Kültür-Sanat',
 1: 'Ekonomi',
 2: 'Siyaset',
 3: 'Eğitim',
 4: 'Dünya',
 5: 'Spor',
 6: 'Teknoloji',
 7: 'Magazin',
 8: 'Sağlık',
 9: 'Gündem'}

# Prediction Result

In [32]:
#  model prediction
result = model.predict(news_featured)
print(result)
print(labels[result[0]])

[1]
Ekonomi


### Sample 2 

In [33]:
news = "PSG'nin dünyaca ünlü yıldız futbolcusu Neymar, sakatlığı sebebiyle 4 hafta sahalardan uzak kalacak. Neymar'ın son yıllarda kız kardeşi Rafaella Santos'un doğum gününe denk gelen tarihlerde sakatlanması dikkat çekiyor..."

cleaned_news = clean_text(news)
cleaned_news

'psg dünyaca ünlü yıldız futbolcusu neymar sakatlığı sebebiyle sahalardan uzak kalacak neymar yıllarda kız kardeşi rafaella santos doğum gününe denk gelen tarihlerde sakatlanması dikkat çekiyor'

In [34]:
#  preprocessing
word_transformed_news = word_tfidf.transform([cleaned_news])
char_transformed_news = char_tfidf.transform([cleaned_news])
news_featured = hstack([word_transformed_news, char_transformed_news])

#  model prediction
result = model.predict(news_featured)
print(result)
print(labels[result[0]])

[5]
Spor
